In [ ]:
from constraint import ScheduleConfig, ScheduleModel
import pandas as pd
pd.set_option("display.max_columns", None)  # 예: 200개까지
pd.set_option("display.width", 2000)


"""메인 함수"""
print("스케줄링 모델 초기화 중...")


num_nurses =10
config = ScheduleConfig(cycle_days=28)
model = ScheduleModel(num_nurses, config)

# 변수 생성
model.create_variables()

# 주휴 설정 (최우선 조건 - 반드시 먼저 설정되어야 함)
# 주휴는 고정되며 다른 제약보다 우선 적용됨
# {nurse_id: day_of_week} 형식 (day_of_week: 0=일요일, 1=월요일, ..., 6=토요일)
model.set_weekly_rest_days(
    {0: 1, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5, 9: 5}
)

# 하드 제약 추가 (주휴 설정 이후에 적용)
model.add_hard_constraints()


# 모델 풀이
print("모델 풀이 중...")
solution = model.solve(time_limit_seconds=60)

week_map={
    0:"일",
    1:"월",
    2:"화",
    3:"수",
    4:"목",
    5:"금",
    6:"토"
}

if solution:
    print("\n해를 찾았습니다!")
    df = pd.DataFrame(
        {
            f"P{nurse+1}": [solution[nurse][day].name for day in range(config.cycle_days)]
            for nurse in range(num_nurses)
        },
        index=[f"{week_map[day%7]}" for day in range(config.cycle_days)],
    )
    
    if "df" in dir() and df is not None:
        count_d = df.apply(lambda col: (col == "D").sum())
        count_e = df.apply(lambda col: (col == "E").sum())
        count_n = df.apply(lambda col: (col == "N").sum())
        count_o = df.apply(lambda col: ((col == "O") | (col == "WR")).sum())
        cnt = pd.DataFrame({"OFF": count_o, "D": count_d, "E": count_e, "N": count_n})
    
        result = df.merge(cnt, left_index=True, right_index=True).T
else:
    print("\n해를 찾지 못했습니다. 제약조건을 확인해주세요.")


스케줄링 모델 초기화 중...
모델 풀이 중...

해를 찾았습니다!


,일,월,화,수,목,금,토,일,월,화,수,목,금,토,일,월,화,수,목,금,토,일,월,화,수,목,금,토
P1,E,WR,O,D,D,E,E,WR,O,O,O,E,N,N,N,O,O,D,E,E,WR,D,D,D,N,N,WR,O
P2,E,WR,D,E,E,E,O,WR,E,N,N,N,O,O,O,O,D,D,D,D,WR,N,N,N,O,O,WR,D
P3,D,E,WR,N,N,N,O,O,WR,D,D,D,E,E,WR,D,E,N,N,O,O,O,O,O,E,E,E,WR
P4,D,E,WR,N,N,O,O,E,WR,E,O,O,O,O,WR,N,N,N,O,O,D,D,E,E,O,D,D,WR
P5,O,D,E,WR,D,N,N,N,O,WR,E,E,N,N,N,WR,O,O,O,O,E,WR,D,O,D,D,D,E
P6,N,N,N,WR,O,O,O,D,E,WR,N,N,O,O,D,WR,D,O,E,N,N,WR,O,D,D,E,E,E
P7,O,D,D,D,WR,D,N,N,N,O,WR,D,E,E,E,E,WR,E,O,O,D,E,WR,N,N,O,O,O
P8,O,O,O,O,WR,D,E,E,N,N,WR,O,D,D,D,D,WR,E,N,N,O,O,WR,E,E,N,N,N
P9,O,O,E,E,E,WR,D,D,D,D,D,WR,O,O,O,E,E,WR,D,D,N,N,N,WR,O,O,N,N
P10,N,N,N,O,O,WR,D,O,D,E,E,WR,D,D,E,N,N,WR,O,E,E,E,E,WR,O,O,O,D
